# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [55]:

# initial imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv
import plotly.graph_objects as go
from ipywidgets import widgets
from panel.interact import interact
import panel as pn
%matplotlib inline

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')



In [56]:
# Read the Mapbox API key
mapbox_token = os.getenv("MAPBOX_API_KEY")

# Import Data

In [57]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
hood_lst=sfo_data['neighborhood'].unique().tolist()

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [58]:
# Define Panel Visualization Functions
def housing_units_per_year(sfo_data):
    sfo_data_units=sfo_data['housing_units'].groupby(['year']).mean()
    min_y=sfo_data_units.min()
    max_y=sfo_data_units.max()
    std_y=sfo_data_units.std()
    housing_year=sfo_data_units.hvplot.bar(
        ylim=(min_y-std_y, max_y+std_y),
        xlabel='Year', ylabel='Housing units',
        hover=True,
        yformatter="$%f",
        label="Average numbers housing units",
        frame_width=300, frame_height=150,rot=90)
    
    return housing_year
    

def average_gross_rent(sfo_data):
    sfo_data_rent=sfo_data['gross_rent'].groupby(['year']).mean()
    sfo_rent=sfo_data_rent.hvplot(xlabel='Year', ylabel='Gross Rent (USD)',
        hover=True, 
        label="Average gross rent",
        frame_width=300, frame_height=150,rot=90)
    
    return sfo_rent

def average_sales_price(sfo_data):
    sfo_data_price=sfo_data['sale_price_sqr_foot'].groupby(['year']).mean()
    sfo_price=sfo_data_price.hvplot(xlabel='Year', ylabel='Sales Price (USD)',
        hover=True, 
        label="Average sales price",
        frame_width=300, frame_height=150,rot=90)
    
    return sfo_price

def average_price_by_neighborhood(sfo_data):
    sfo_data_hood=sfo_data.groupby(['year','neighborhood']).mean()
    sfo_data_hood.reset_index(inplace=True)
    sfo_hood=sfo_data_hood.hvplot.line(
        x='year', y='sale_price_sqr_foot',
        xlabel='Year', ylabel='Sales Price (USD)',
        hover=True, groupby='neighborhood',
        label="Selling price per year for selected neighborhoods",
        frame_width=300, frame_height=150,rot=90)
    
    return sfo_hood

def top_most_expensive_neighborhoods(sfo_data):
    sf_data_hood_large=sfo_data.groupby(by='neighborhood').mean().nlargest(10,'sale_price_sqr_foot')
    sf_data_hood_large.reset_index(inplace=True)
    min_y=sf_data_hood_large['sale_price_sqr_foot'].min()
    max_y=sf_data_hood_large['sale_price_sqr_foot'].max()
    std_y=sf_data_hood_large['sale_price_sqr_foot'].std()
    ylim=(min_y-std_y, max_y+std_y),
    
    sfo_largest=sf_data_hood_large.hvplot.bar(
        x='neighborhood',y='sale_price_sqr_foot',
        xlabel='Neighborhood', ylabel='Sales Price (USD)',
        hover=True, 
        label="Selling price of most expensive SF neighborhoods",
        frame_width=600, frame_height=300,rot=90)
    
    return sfo_largest


def parallel_coordinates(sfo_data):
    sf_data_hood_large=sfo_data.groupby(by='neighborhood').mean().nlargest(10,'sale_price_sqr_foot')
    sf_data_hood_large.reset_index(inplace=True)
    sfo_coord=px.parallel_coordinates(
        sf_data_hood_large,
        dimensions=['neighborhood',"sale_price_sqr_foot", "housing_units", "gross_rent"],
        color="housing_units",
        color_continuous_scale=px.colors.diverging.Portland,
        labels={'neighborhood':"Neighborhood",'sale_price_sqr_foot':'Sales Price (USD)','housing_units':'Units','gross_rent':'Rents'}
    )
    return sfo_coord

def parallel_categories(sfo_data):
    sf_data_hood_large=sfo_data.groupby(by='neighborhood').mean().nlargest(10,'sale_price_sqr_foot')
    sf_data_hood_large.reset_index(inplace=True)
    sf_data_hood_large['sale_price_sqr_foot']=sf_data_hood_large['sale_price_sqr_foot'].astype('int')
    sf_data_hood_large['gross_rent']=sf_data_hood_large['gross_rent'].astype('int')
    sf_data_hood_large.dtypes
    sfo_cat=px.parallel_categories(sf_data_hood_large,
            dimensions=['neighborhood',"sale_price_sqr_foot", "housing_units", "gross_rent"], 
            color="sale_price_sqr_foot",
            color_continuous_scale=px.colors.sequential.Rainbow,
            labels={'neighborhood':"Neighborhood",'sale_price_sqr_foot':'Sales Price (USD)','housing_units':'Units','gross_rent':'Rents'})



    return sfo_cat


def neighborhood_map(sfo_data, df_neighborhood_locations,mapbox_token):
    px.set_mapbox_access_token(mapbox_token)

    df_neighborhood_locations.set_index('Neighborhood', inplace=True)
    hood_mean=sfo_data.groupby('neighborhood').mean()
    hood_mean=pd.concat([hood_mean,df_neighborhood_locations], axis=1, join='inner')
    price_map=px.scatter_mapbox(
        hood_mean,
        lat="Lat",
        lon="Lon",
        hover_name=hood_mean.index,
        size="sale_price_sqr_foot",
        color="gross_rent",
        color_continuous_scale=px.colors.cyclical.Edge,
        zoom=10,
        title=f'SF Housing cost by neighborhood',
        width=1000,
    )
    price_map
    return price_map



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [59]:
housing=housing_units_per_year(sfo_data)
gross_rents=average_gross_rent(sfo_data)
sales_price=average_sales_price(sfo_data)
sfo_hood=average_price_by_neighborhood(sfo_data)
sfo_largest=top_most_expensive_neighborhoods(sfo_data)
sfo_coord=parallel_coordinates(sfo_data)
sfo_cat=parallel_categories(sfo_data)
price_map=neighborhood_map(sfo_data, df_neighborhood_locations,mapbox_token)

In [76]:
yearly_row1 = pn.Row(sfo_hood, sales_price)
yearly_row2 = pn.Row(gross_rents, housing)

intro = pn.Column("##Housing rends analysis for selected San Francisco neighborhoods", 
                  """In support of Harold's efforts to relocate his parents we have undertaken 
                  an analyis of the nighborhoods in Sanfrancisco, focusing on the trends in sale cost, 
                  rents and available inventory.  The results are showing in the following graphs:""", 
                  '''* Yearly Trends\n
    - Yearly increas in sale price by neighborhood\n
    - Average yearly increas in sale price by year\n
    - Average gross rents by year\n
    - Average number of units by year\n
* Yearly Trends\n
    - Average trends by coordinate\n
    - Average trends by category \n
* Neighborhood Trends\n
    - Average sale price by physical location \n
    - Top 10 most expensive San Francisco neighborhoods''', width=600)   

yearly = pn.Column("#### Trends by year", yearly_row1, yearly_row2)


parallel = pn.Column("#### Multidimensional trends",sfo_coord, sfo_cat)


hoods = pn.Column("#### Price by neighborhood", price_map, sfo_largest)






sfo_housing_market = pn.Tabs(
("Introduction", intro),
("Yearly Trends", yearly),
('Parallel Correlation', parallel),
('Trends by Neighborhood', hoods)
)

sfo_housing_market

Tabs
    [0] Column(width=600)
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
            [1] HoloViews(Curve)
        [2] Row
            [0] HoloViews(Curve)
            [1] HoloViews(Bars)
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] HoloViews(Bars)

In [12]:
    df_neighborhood_locations.set_index('Neighborhood', inplace=True)
    hood_mean=sfo_data.groupby('neighborhood').mean()
    hood_mean=pd.concat([hood_mean,df_neighborhood_locations], axis=1, join='inner')
    price_map=px.scatter_mapbox(
        hood_mean,
        lat="Lat",
        lon="Lon",
        hover_name=hood_mean.index,
        size="sale_price_sqr_foot",
        color="gross_rent",
        color_continuous_scale=px.colors.cyclical.Edge,
        zoom=10,
        title=f'SF Housing cost by neighborhood',
        width=1000,
    )
    price_map

## Serve the Panel Dashboard

In [ ]:
sfo_housing _market.servable()

In [ ]:
housing

In [ ]:
gross_rents

In [ ]:
sales_price

In [ ]:
sfo_hood

In [62]:
sfo_largest

:Bars   [neighborhood]   (sale_price_sqr_foot)

In [64]:
sfo_coord

In [63]:
sfo_cat

In [61]:
price_map

In [ ]:

housing=housing_units_per_year(sfo_data)
gross_rents=average_gross_rent(sfo_data)
sales_price=average_sales_price(sfo_data)
sfo_hood=average_price_by_neighborhood(sfo_data)
sfo_largest=top_most_expensive_neighborhoods(sfo_data)
sfo_coord=parallel_coordinates(sfo_data)
sfo_cat=parallel_categories(sfo_data)
price_map=neighborhood_map(sfo_data, df_neighborhood_locations,mapbox_token)

In [68]:
 pn.Column?
    

Init signature:
pn.Column(
    *objects,
    ncols,
    nrows,
    scroll,
    align,
    aspect_ratio,
    background,
    css_classes,
    height,
    height_policy,
    margin,
    max_height,
    max_width,
    min_height,
    min_width,
    sizing_mode,
    width,
    width_policy,
    name,
    **kwargs,
)
Docstring:     
    Vertical layout of Viewables.
    
Parameters of 'Column'

Parameters changed from their default values are marked in red.
Soft bound values are marked in cyan.
C/V= Constant/Variable, RO/RW = ReadOnly/ReadWrite, AN=Allow None

Name             Value        Type         Bounds     Mode  

align           'start'  ObjectSelector               V RW  
aspect_ratio      None     Parameter                V RW AN 
background        None     Parameter                V RW AN 
css_classes       None        List       (0, None)  V RW AN 
height            None      Integer      (0, None)  V RW AN 
height_policy    'auto'  ObjectSelector               V RW  
margin    